# 在使用 AWS SDK 前，須安裝以下套件 

##### Software Development Kit（SDK）【軟體開發套件】
###### * Boto3（AWS 的 Python 套件）
###### * AWS CLI（Command Line Interface）

### 安裝套件

In [1]:
# 安裝套件
!pip install boto3
!pip install awscli

  Using cached https://files.pythonhosted.org/packages/09/c6/d3e3abe5b4f4f16cf0dfc9240ab7ce10c2baa0e268989a4e3ec19e90c84e/urllib3-1.26.4-py2.py3-none-any.whl
  Found existing installation: urllib3 1.24.2
    Uninstalling urllib3-1.24.2:
      Successfully uninstalled urllib3-1.24.2


ERROR: conda 4.10.0 requires ruamel_yaml_conda>=0.11.14, which is not installed.
ERROR: requests 2.22.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.4 which is incompatible.


In [2]:
# 其它套件
## SQL 資料庫套件
!pip install mysql-connector-python #與SQL進行溝通

### 匯入套件

In [61]:
# 匯入套件
from pprint import pprint  #優化輸出，例如：pandas的value.tolist輸出不是一行，而是一個一個精美的排列，就是pprint的作用。
from decimal import Decimal #小數的相關處理
from botocore.exceptions import ClientError
import boto3
import mysql.connector #與SQL進行溝通
import json

### 設定Token

#### AWS Educate

In [83]:
# Token 設定  - AWS Educate
## 若您的程式執行於 AWS 環境（如：EC2），無需進行設定，系統將自動完成驗證動作。
## 若您執行於 Local 端（自己的筆電），請完成該步驟。
aws = {
    'aws_access_key_id' : '',
    'aws_secret_access_key' : '',
    'aws_session_token' : ''
}
# *** AWS 驗證金鑰可至 AWS Educate Workbench 底下的【Account Details】Button 中的 AWS CLI 區間取得該金鑰資訊。
#     由於金鑰具有 3 小時的時效性，因此截止時間到了之後須重新複製貼上。

#### AWS 付費帳號

In [5]:
#會比前述的 AWS Educate 少一個session token
aws = {
    'aws_access_key_id' : '',
    'aws_secret_access_key' : ''
}

# **案例 1：【S3】上傳 logo_SM.jpg**

#### Boto3 documentation：https://reurl.cc/1gRde8
#### Boto3 - S3：https://reurl.cc/bzpLE3

### STEP 1：透過配置好的 Token 資訊，進行 AWS S3 服務的連接

In [84]:
s3 = boto3.resource(
        's3',
        aws_access_key_id = aws['aws_access_key_id'],
        aws_secret_access_key = aws['aws_secret_access_key'],
        aws_session_token = aws['aws_session_token']
    )
#要用resource還是client可以參照document，但前者的權限更高，可以取得更多資料。
#若是先使用client，發現被阻擋，可以考慮更換成resource。

### STEP 2：藉由迴圈，獲取 S3 Bucket 的所有列表

In [85]:
for bucket in s3.buckets.all():
    print(bucket.name)

aws-hw3-s3-amplify


### STEP 3：在獲取 S3 Bucket List 後，選擇要上傳文件至該 Bucket

In [86]:
filepath = r'C:\Users\USER\Desktop\AWS_HW\AWS_3_S3_Amplify\table_demo\logo_SM.jpg'
data = open(filepath, 'rb')
s3.Bucket('aws-hw3-s3-amplify').put_object(Key='logo_SM.jpg', Body=data)
## Key ： 上傳後的檔名
## Body： 上傳的資料（一堆 Bytes）

s3.Object(bucket_name='aws-hw3-s3-amplify', key='logo_SM.jpg')

# **案例 2：【RDS】完成 CRUD 操作**

## 連接 AWS RDS

In [87]:
# Configuration Database
cnx = mysql.connector.connect(
    host = "",
    user = "",
    password = ""
)

print(cnx)

In [88]:
# Cursor Ready
cursor = cnx.cursor()

## CRUD【AWS RDS】 操作

### 1.Create Syntax【AWS RDS】

### STEP 1-1：Create Database

In [89]:
sql = """
    CREATE DATABASE aws_hw5_db_rds;
"""
print('CREATE DATABASE:',cursor.execute(sql)) #執行：cursor.execute

# Output：
# CREATE DATABASE: None--->興建成功

CREATE DATABASE: None


#### **備註：database興建好後，要再重新連線一次。故cnx內要多增加一個參數database，因為要確認是在哪個指定的database下進行後續的資料庫行為。**

In [91]:
# 重新連線，有指定的database。
# Configuration Database
cnx = mysql.connector.connect(
    host = "",
    user = "",
    password = "",
    database = ''
)

print(cnx)

# Cursor Ready
cursor = cnx.cursor()

### STEP 1-2：Create Table

In [92]:
#Table資料簡析
# Table Name：Persons
# Table Column：
#     1.PersonID ---?int
#     2.LastName --->varchar(255)
#     3.FirstName --->varchar(255)
#     4.Address --->varchar(255)
#     5.City --->varchar(255)
sql = """
    CREATE TABLE Persons (
        PersonID int,
        LastName varchar(255),
        FirstName varchar(255),
        Address varchar(255),
        City varchar(255)
    );
"""
print('CREATE TABLE:',cursor.execute(sql))

# Output：
# CREATE TABLE: None--->興建成功

CREATE TABLE: None


### STEP 1-3：Show Tables

In [93]:
sql = """
    SHOW TABLES;
"""
cursor.execute(sql)
tables = cursor.fetchall()
print('SHOW TABLES:',tables)

# Output：
# SHOW TABLES: [('Persons',)]--->DB中確實有Persons這張table

SHOW TABLES: [('Persons',)]


### STEP 1-4：Insert data

In [94]:
# 新增單筆資料
sql = """
    INSERT INTO Persons(PersonID, LastName, FirstName, Address, City)
    VALUES(%s, %s, %s, %s, %s)
"""
#在Insert Values中標準是用%s作為每個欄位中的切割值，若是用其他format string過後的值一樣可以輸入，只是會有SQL Injection的惡意程式植入的風險。

values = (1, 'Cindy', 'Hung', '臺北市士林區臨溪路70號', '臺北市') 
#雖然前述在table instert時限定value欄位型態，但是可以經由value新增的值進行修正，而value是用一個tuple作為一筆資料(row)。
#一開始定義多少個欄位，新增的資料欄位就要有幾個，意指一個tuple內的元素個數會等於興建table時的欄位個數。
cursor.execute(sql, values)
cnx.commit() #確認上傳
print('INSERT TABLES:',cursor.rowcount) #rowcount更動幾筆資料

# Output：
# INSERT TABLES: 1---> DB中確實有新增1筆資料進入Persons這張table中

INSERT TABLES: 1


In [95]:
##若是要新增多筆資料，那就會使用list of tuple的方式進行新增。--->[(tupl1), (tuple2), ....]
sql = """
    INSERT INTO Persons(PersonID, LastName, FirstName, Address, City)
    VALUES(%s, %s, %s, %s, %s)
"""
values_list = []
t1 = (2, 'Jenny', 'Lin', '臺北市士林區臨溪路70號', '臺北市') 
t2 = (3, 'Zoe', 'Chen', '彰化縣彰化市南校街135號', '彰化縣') 
t3 = (4, 'Cathy', 'Huang', '臺北市中正區貴陽街一段56號', '臺北市') 
# values_list.append(t1)
# values_list.append(t2)
# values_list.append(t3)
values_list = [t1, t2, t3]
print(values_list)
cursor.executemany(sql, values_list) #批量上傳
cnx.commit()
print('INSERT TABLES:',cursor.rowcount)
# Output：
# INSERT TABLES: 3---> DB中確實有新增3筆資料進入Persons這張table中

[(2, 'Jenny', 'Lin', '臺北市士林區臨溪路70號', '臺北市'), (3, 'Zoe', 'Chen', '彰化縣彰化市南校街135號', '彰化縣'), (4, 'Cathy', 'Huang', '臺北市中正區貴陽街一段56號', '臺北市')]
INSERT TABLES: 3


### 2.Read Syntax【AWS RDS】

In [96]:
# SELECT查詢
sql = """
    SELECT * FROM Persons
    WHERE Persons.PersonID = 1;
"""
cursor.execute(sql)
for each in cursor:
    print('SELECT DATA:',each)

#計算回傳資料總長度時，可不必用 pythoon len 來計算，直接套用 SQL 語法中的 COUNT 完成。

SELECT DATA: (1, 'Cindy', 'Hung', '臺北市士林區臨溪路70號', '臺北市')


###### *** 若您在執行 UPDATE 及 DELETE 的過程中發生 Error Code 1175
######     試著輸入 【SET SQL_SAFE_UPDATES = 0;】 在 SQL 語法前，
######     因為有些 Workbrench 預設將禁止 UPDATE 及 DELETE 行為，
######     0 為關閉禁止，1 為開啟功能。
######     若您想要直接在畫面上完成解除禁止的功能，可以跟著以下步驟:
######     (僅適用與 MySQL Workbrench)
######     1. Go to Edit --> Preferences
######     2. Click "SQL Editor" tab and uncheck "Safe Updates" check box
######     3. Query --> Reconnect to Server // logout and then login
######     4. Now execute your SQL query

### 3.Update Syntax【AWS RDS】

In [97]:
sql = """
    INSERT INTO Persons(PersonID, LastName, FirstName, Address, City)
    VALUES(%s, %s, %s, %s, %s)
"""


values = (1, 'Joe', 'Wu', '新竹市東區力行六路8號', '新竹市') 

cursor.execute(sql, values)
cnx.commit() #確認上傳
print('INSERT TABLES:',cursor.rowcount) #rowcount更動幾筆資料

INSERT TABLES: 1


In [102]:
# UPDATE
sql = """
    UPDATE Persons 
    SET PersonID = %s WHERE PersonID = %s;
"""
values = (5,1)
cursor.execute(sql,values)
cnx.commit() #確認上傳
print('UPDATE DATA:',cursor.rowcount)
# DUPLICATE KEY UPDATE

UPDATE DATA: 0


In [99]:
# 再次SELECT查詢
sql = """
    SELECT * FROM Persons
    WHERE Persons.PersonID = 5;
"""
cursor.execute(sql)
for each in cursor:
    print('SELECT DATA:',each)

SELECT DATA: (5, 'Cindy', 'Hung', '臺北市士林區臨溪路70號', '臺北市')
SELECT DATA: (5, 'Joe', 'Wu', '新竹市東區力行六路8號', '新竹市')


### 4.Delete Syntax【AWS RDS】

In [100]:
## DELETE
sql = """
    DELETE FROM Persons WHERE PersonID = 3;
"""
cursor.execute(sql)
cnx.commit() #確認上傳
print('DELETE DATA:',cursor.rowcount)

# AUTO_INCREMENT = 100--->從100開始自動編號，若後面沒有寫起始編號，預設為1。

## AUTO_INCREMENT流水編號下的兩種刪除的差異(假設刪除前資料有 7 筆)：
## DROP TABLE：不是把table整個刪掉，而是把table內的資料全部刪除，但欄位都會保留下來。若是新增一筆資料進去後，流水編號則從 8 開始繼續編號。
## TRUNCATE TABLE：移除table中的所有資料列，但會保留資料表結構及其欄位、條件約束、索引等。若是新增一筆資料進去後，流水編號則從 1 開始重新編號。

DELETE DATA: 1


In [101]:
# 再次SELECT查詢
sql = """
    SELECT * FROM Persons
    WHERE Persons.PersonID = 3;
"""
cursor.execute(sql)
for each in cursor:
    print('SELECT DATA:',each)

## 啟動 / 終結 Instance

#### 注意：由於DB重新啟動需要近5分鐘的時間，所以不建議高頻率的開關Instance。

### 補充 STEP 1：透過配置好的 Token 資訊，進行 AWS RDS 服務的連接

In [ ]:
rds = boto3.client(
        'rds',
        aws_access_key_id = aws['aws_access_key_id'],
        aws_secret_access_key = aws['aws_secret_access_key'],
        aws_session_token = aws['aws_session_token']
    )

### 補充 STEP 2: 啟動 db instance

In [ ]:
#DBInstanceIdentifier中填上想要在RDS中讀取的DB名稱
response = rds.start_db_instance(
    DBInstanceIdentifier = 'aws-cc-ft'
)

### 補充 STEP 3: 終結 db instance

In [ ]:
response = rds.stop_db_instance(
    DBInstanceIdentifier = 'aws-cc-ft'
)

# **案例 3：【DynamoDB】完成 CRUD 操作**

### DynamoDB documentation：https://reurl.cc/o9GWQv

## 連接 DynamoDB

In [103]:
# 透過配置好的 Token 資訊，進行 AWS DynamoDB 服務的連接
#要先新增一個參數region_name，可以至aws educate查詢帳號所在區域，由於aws educate的使用區域限定為N.Virginia，故region_name 為 'us-east-1'。
dynamodb = boto3.resource(
        'dynamodb',
        aws_access_key_id = aws['aws_access_key_id'],
        aws_secret_access_key = aws['aws_secret_access_key'],
        aws_session_token = aws['aws_session_token'],
        region_name = 'us-east-1'
    )

## CRUD【DynamoDB】 操作

### 1.Create Syntax【DynamoDB】

### STEP 1-1：Create Table https://reurl.cc/YWzydD

In [104]:
def create_movie_table(dynamodb=None): #若是要用外網連接，就需要附加dynamodb的參數
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.create_table(
        TableName='Movies',
        KeySchema=[
            {
                'AttributeName': 'year',
                'KeyType': 'HASH'  # Partition key 唯一值
            },
            {
                'AttributeName': 'title',
                'KeyType': 'RANGE'  # Sort key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'year',
                'AttributeType': 'N' # N：number
            },
            {
                'AttributeName': 'title',
                'AttributeType': 'S' # S：string
            },

        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )
    return table


if __name__ == '__main__':
    movie_table = create_movie_table(dynamodb)
    print("Table status:", movie_table.table_status)

Table status: CREATING


### STEP 1-2：Load Data(moviedata.json)

In [113]:
def load_movies(movies, dynamodb=None): 
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.Table('Movies')
    for movie in movies:
        year = int(movie['year'])
        title = movie['title']
        print("Adding movie:", year, title)
        table.put_item(Item=movie)


if __name__ == '__main__':
    with open(r"C:\Users\USER\Desktop\AWS_HW\AWS_5_DatabaseServices_RDS_DynamoDB\moviedata.json") as json_file:
        movie_list = json.load(json_file, parse_float=Decimal)
    load_movies(movie_list,dynamodb)

Adding movie: 2013 Rush
Adding movie: 2013 Prisoners
Adding movie: 2013 The Hunger Games: Catching Fire
Adding movie: 2013 Thor: The Dark World
Adding movie: 2013 This Is the End
Adding movie: 2013 Insidious: Chapter 2
Adding movie: 2013 World War Z
Adding movie: 2014 X-Men: Days of Future Past
Adding movie: 2014 Transformers: Age of Extinction
Adding movie: 2013 Now You See Me
Adding movie: 2013 Gravity
Adding movie: 2013 We're the Millers
Adding movie: 2013 Riddick
Adding movie: 2013 The Family
Adding movie: 2013 Star Trek Into Darkness
Adding movie: 2013 After Earth
Adding movie: 2013 The Great Gatsby
Adding movie: 2014 Divergent
Adding movie: 2013 We Are What We Are
Adding movie: 2013 Iron Man 3
Adding movie: 2014 The Amazing Spider-Man 2
Adding movie: 2013 Curse of Chucky
Adding movie: 2013 The Conjuring
Adding movie: 2013 Oldboy
Adding movie: 2013 Escape Plan
Adding movie: 2013 Elysium
Adding movie: 2013 Cloudy with a Chance of Meatballs 2
Adding movie: 2014 RoboCop
Adding movie:

Adding movie: 2013 Paranoia
Adding movie: 2012 Jagten
Adding movie: 1998 The Big Lebowski
Adding movie: 2000 Remember the Titans
Adding movie: 1987 The Princess Bride
Adding movie: 1982 Blade Runner
Adding movie: 2013 Austenland
Adding movie: 1993 Jurassic Park
Adding movie: 2013 C.O.G.
Adding movie: 1995 Se7en
Adding movie: 2010 The Town
Adding movie: 2013 Gangster Squad
Adding movie: 2012 Rock of Ages
Adding movie: 2001 Zoolander
Adding movie: 2003 Finding Nemo
Adding movie: 2009 (500) Days of Summer
Adding movie: 1966 Il buono, il brutto, il cattivo.
Adding movie: 2012 Life of Pi
Adding movie: 2013 The Big Wedding
Adding movie: 1999 The Matrix
Adding movie: 2014 Vampire Academy
Adding movie: 2010 Iron Man 2
Adding movie: 1994 Forrest Gump
Adding movie: 2013 Parker
Adding movie: 2004 The Chronicles of Riddick
Adding movie: 1994 The Lion King
Adding movie: 2012 Jack Reacher
Adding movie: 2011 The Girl with the Dragon Tattoo
Adding movie: 2009 Avatar
Adding movie: 2012 End of Watch
Add

Adding movie: 1999 The Sixth Sense
Adding movie: 2012 LOL
Adding movie: 2001 Monsters, Inc.
Adding movie: 2012 The Bourne Legacy
Adding movie: 2010 TRON: Legacy
Adding movie: 2007 Juno
Adding movie: 2013 La migliore offerta
Adding movie: 2011 Fright Night
Adding movie: 2010 Machete
Adding movie: 2003 Lost in Translation
Adding movie: 2013 August: Osage County
Adding movie: 2008 The Incredible Hulk
Adding movie: 2012 To the Wonder
Adding movie: 1998 Mulan
Adding movie: 2009 District 9
Adding movie: 2012 The Man with the Iron Fists
Adding movie: 2006 El laberinto del fauno
Adding movie: 2012 Here Comes the Boom
Adding movie: 2012 Taken 2
Adding movie: 2012 The First Time
Adding movie: 2014 Hercules: The Thracian Wars
Adding movie: 2013 A Good Day to Die Hard
Adding movie: 2004 Crash
Adding movie: 2010 The Other Guys
Adding movie: 2011 Rio
Adding movie: 2008 Gran Torino
Adding movie: 1978 Animal House
Adding movie: 1986 Ferris Bueller's Day Off
Adding movie: 2012 Parental Guidance
Adding 

Adding movie: 2002 The Count of Monte Cristo
Adding movie: 1998 The Truman Show
Adding movie: 2013 Legends of Oz: Dorothy's Return
Adding movie: 2013 Paradise
Adding movie: 2006 Mission: Impossible III
Adding movie: 2004 EuroTrip
Adding movie: 2014 Veronica Mars
Adding movie: 2004 The Passion of the Christ
Adding movie: 2003 The Italian Job
Adding movie: 2012 The Dictator
Adding movie: 2012 Contraband
Adding movie: 2001 Pearl Harbor
Adding movie: 2004 Der Untergang
Adding movie: 2010 Winter's Bone
Adding movie: 2002 Insomnia
Adding movie: 1997 The Lost World: Jurassic Park
Adding movie: 2013 Snowpiercer
Adding movie: 2012 Truth or Dare
Adding movie: 2001 Moulin Rouge!
Adding movie: 2012 Trouble with the Curve
Adding movie: 1988 Child's Play
Adding movie: 2008 Slumdog Millionaire
Adding movie: 2012 Wrath of the Titans
Adding movie: 1982 Poltergeist
Adding movie: 2012 What Maisie Knew
Adding movie: 2012 Safe House
Adding movie: 2012 The Expatriate
Adding movie: 2012 Mirror Mirror
Adding 

Adding movie: 2009 Pandorum
Adding movie: 2005 Constantine
Adding movie: 2009 2012
Adding movie: 2008 Seven Pounds
Adding movie: 2009 Man som hatar kvinnor
Adding movie: 2008 Definitely, Maybe
Adding movie: 2011 Like Crazy
Adding movie: 2007 Death Proof
Adding movie: 2004 The Bourne Supremacy
Adding movie: 2005 Madagascar
Adding movie: 2012 Beasts of the Southern Wild
Adding movie: 2006 Rocky Balboa
Adding movie: 2012 Alex Cross
Adding movie: 2014 Transcendence
Adding movie: 2005 The New World
Adding movie: 2003 The Last Samurai
Adding movie: 1980 The Elephant Man
Adding movie: 2012 People Like Us
Adding movie: 1964 Mary Poppins
Adding movie: 2005 King Kong
Adding movie: 2000 Gone in Sixty Seconds
Adding movie: 2014 Tomorrowland
Adding movie: 2004 Before Sunset
Adding movie: 2004 Friday Night Lights
Adding movie: 2011 J. Edgar
Adding movie: 2008 The Boy in the Striped Pyjamas
Adding movie: 2002 Signs
Adding movie: 2000 Road Trip
Adding movie: 2005 Hitch
Adding movie: 2009 Brothers
Addi

...

Adding movie: 2009 The Messenger
Adding movie: 2010 Tropa de Elite 2 - O Inimigo Agora E Outro
Adding movie: 2013 No se si cortarme las venas o dejarmelas largas
Adding movie: 2011 The Tunnel
Adding movie: 1999 Life
Adding movie: 2007 The Reaping
Adding movie: 1998 Mercury Rising
Adding movie: 2009 I Hope They Serve Beer in Hell
Adding movie: 1987 Superman IV: The Quest for Peace
Adding movie: 2008 What Just Happened
Adding movie: 1991 The Commitments
Adding movie: 1985 Friday the 13th: A New Beginning
Adding movie: 2009 Planet 51
Adding movie: 2003 Saints and Soldiers
Adding movie: 2006 Arthur et les Minimoys
Adding movie: 2002 Life or Something Like It
Adding movie: 2009 Bakjwi
Adding movie: 1995 Johnny Mnemonic
Adding movie: 2000 Ready to Rumble
Adding movie: 2005 Miss Congeniality 2: Armed and Fabulous
Adding movie: 1995 It Takes Two
Adding movie: 1973 High Plains Drifter
Adding movie: 1958 Touch of Evil
Adding movie: 2001 Le pacte des loups
Adding movie: 2011 Kill the Irishman
Add

Adding movie: 2005 The Weather Man
Adding movie: 1995 La cite des enfants perdus
Adding movie: 2006 Facing the Giants
Adding movie: 2000 The Skulls
Adding movie: 2012 Asterix et Obelix: Au service de Sa Majeste
Adding movie: 2010 Lottery Ticket
Adding movie: 1997 An American Werewolf in Paris
Adding movie: 1955 The Night of the Hunter
Adding movie: 1994 In the Mouth of Madness
Adding movie: 2011 Wrong Turn 4: Bloody Beginnings
Adding movie: 2002 Dirty Pretty Things
Adding movie: 2006 Huo Yuan Jia
Adding movie: 2013 Jimmy P.
Adding movie: 1968 Where Eagles Dare
Adding movie: 1999 Superstar
Adding movie: 2008 Cadillac Records
Adding movie: 2014 Jane Got a Gun
Adding movie: 1995 First Knight
Adding movie: 1984 Paris, Texas
Adding movie: 2012 Come Out and Play
Adding movie: 1986 The Money Pit
Adding movie: 2007 A Mighty Heart
Adding movie: 1985 European Vacation
Adding movie: 2004 Anacondas: The Hunt for the Blood Orchid
Adding movie: 2010 The Shrine
Adding movie: 2013 In Fear
Adding movie

Adding movie: 2010 Wrecked
Adding movie: 2008 One Missed Call
Adding movie: 2002 Femme Fatale
Adding movie: 2009 Post Grad
Adding movie: 2013 Hoe Duur was de Suiker
Adding movie: 1982 The Last Unicorn
Adding movie: 1990 Darkman
Adding movie: 2014 AfterDeath
Adding movie: 1931 Street Scene
Adding movie: 2007 Savage Grace
Adding movie: 1988 Pumpkinhead
Adding movie: 1992 Jennifer Eight
Adding movie: 1994 Blue Sky
Adding movie: 2013 Black Nativity
Adding movie: 2014 Paddington
Adding movie: 2011 Madea's Big Happy Family
Adding movie: 2008 Mutant Chronicles
Adding movie: 1991 Return to the Blue Lagoon
Adding movie: 2010 Cemetery Junction
Adding movie: 2010 Why Did I Get Married Too?
Adding movie: 2007 Arn: Tempelriddaren
Adding movie: 2006 Fast Food Nation
Adding movie: 2013 Absence
Adding movie: 2005 Grizzly Man
Adding movie: 2004 Bride & Prejudice
Adding movie: 2013 Copperhead
Adding movie: 1984 The Natural
Adding movie: 1999 Arlington Road
Adding movie: 1978 Every Which Way But Loose
Ad

Adding movie: 2012 Interview with a Hitman
Adding movie: 1981 Body Heat
Adding movie: 1995 Balto
Adding movie: 1952 The Member of the Wedding
Adding movie: 2010 My Soul to Take
Adding movie: 2007 2 Days in Paris
Adding movie: 2009 The Vicious Kind
Adding movie: 2013 Ass Backwards
Adding movie: 2011 Red Dog
Adding movie: 2014 Love, Rosie
Adding movie: 1989 Loverboy
Adding movie: 2010 The Reef
Adding movie: 2006 Firewall
Adding movie: 2014 The Gunman
Adding movie: 1999 In Dreams
Adding movie: 2006 Running with Scissors
Adding movie: 2012 Spike Island
Adding movie: 2006 Skinwalkers
Adding movie: 2013 Raze
Adding movie: 2002 Divine Secrets of the Ya-Ya Sisterhood
Adding movie: 1984 The Killing Fields
Adding movie: 1989 Leviathan
Adding movie: 1971 Billy Jack
Adding movie: 2008 Happy-Go-Lucky
Adding movie: 1994 Baby's Day Out
Adding movie: 1979 Stalker
Adding movie: 2001 Captain Corelli's Mandolin
Adding movie: 2012 Mac & Devin Go to High School
Adding movie: 2011 Puncture
Adding movie: 201

Adding movie: 1972 The Poseidon Adventure
Adding movie: 2009 Creation
Adding movie: 2004 Fahrenheit 9/11
Adding movie: 1986 The Delta Force
Adding movie: 2012 Wrong
Adding movie: 1982 Halloween III: Season of the Witch
Adding movie: 2011 Rundskop
Adding movie: 1996 The Glimmer Man
Adding movie: 2013 Aashiqui 2
Adding movie: 2007 Anamorph
Adding movie: 2010 Our Family Wedding
Adding movie: 2011 Beneath the Darkness
Adding movie: 1991 Double Impact
Adding movie: 1999 The Other Sister
Adding movie: 2007 I Think I Love My Wife
Adding movie: 2011 La delicatesse
Adding movie: 2011 The Reunion
Adding movie: 1994 Trois couleurs: Rouge
Adding movie: 1995 Total Eclipse
Adding movie: 2012 Freeloaders
Adding movie: 2009 Horsemen
Adding movie: 1995 Welcome to the Dollhouse
Adding movie: 2014 John Wick
Adding movie: 1984 Bachelor Party
Adding movie: 2006 Love and Other Disasters
Adding movie: 1996 Jane Eyre
Adding movie: 1982 The World According to Garp
Adding movie: 1981 Mommie Dearest
Adding movie

### STEP 1-3：Put Data

In [105]:
def put_movie(title, year, plot, rating, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.Table('Movies')
    response = table.put_item(
       Item={
            'year': year,
            'title': title,
            'info': {
                'plot': plot,
                'rating': rating
            }
        }
    )
    return response


if __name__ == '__main__':
    movie_resp = put_movie("The Big New Movie", 2015,
                           "Nothing happens at all.", 0, dynamodb)
    print("Put movie succeeded:")
    #pprint(movie_resp, sort_dicts=False)
    pprint(movie_resp)

Put movie succeeded:
{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '2',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Mon, 03 May 2021 00:29:42 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '2745614147',
                                      'x-amzn-requestid': 'N107UOR868A2QT8RLOBIJQVJSNVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'N107UOR868A2QT8RLOBIJQVJSNVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


### 2.Read Syntax【DynamoDB】

In [106]:
## Read Data
def get_movie(title, year, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.Table('Movies')

    try:
        response = table.get_item(Key={'year': year, 'title': title})
    except ClientError as e:
        print(e.response['Error']['Message'])
    else:
        return response['Item']


if __name__ == '__main__':
    movie = get_movie("The Big New Movie", 2015,dynamodb)
    if movie:
        print("Get movie succeeded:")
        #pprint(movie, sort_dicts=False)
        pprint(movie)

Get movie succeeded:
{'info': {'plot': 'Nothing happens at all.', 'rating': Decimal('0')},
 'title': 'The Big New Movie',
 'year': Decimal('2015')}


### 3.Update Syntax【DynamoDB】

In [107]:
## Update Data
def update_movie(title, year, rating, plot, actors, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.Table('Movies')

    response = table.update_item(
        Key={
            'year': year,
            'title': title
        },
        UpdateExpression="set info.rating=:r, info.plot=:p, info.actors=:a",
        ExpressionAttributeValues={
            ':r': Decimal(rating),
            ':p': plot,
            ':a': actors
        },
        ReturnValues="UPDATED_NEW"
    )
    return response


if __name__ == '__main__':
    update_response = update_movie(
        "The Big New Movie", 2015, 5.5, "Everything happens all at once.",
        ["Larry", "Moe", "Curly"],dynamodb)
    print("Update movie succeeded:")
    #pprint(update_response, sort_dicts=False)
    pprint(update_response)

Update movie succeeded:
{'Attributes': {'info': {'actors': ['Larry', 'Moe', 'Curly'],
                         'plot': 'Everything happens all at once.',
                         'rating': Decimal('5.5')}},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '156',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Mon, 03 May 2021 00:32:21 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '3767510606',
                                      'x-amzn-requestid': 'F0MFSG8B087C8OJK244N6FMHRFVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'F0MFSG8B087C8OJK244N6FMHRFVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


In [108]:
movie = get_movie("The Big New Movie", 2015,dynamodb)
if movie:
    print("Get movie succeeded:")
    #pprint(movie, sort_dicts=False)
    pprint(movie)

Get movie succeeded:
{'info': {'actors': ['Larry', 'Moe', 'Curly'],
          'plot': 'Everything happens all at once.',
          'rating': Decimal('5.5')},
 'title': 'The Big New Movie',
 'year': Decimal('2015')}


### 4.Delete Syntax【DynamoDB】

In [109]:
## Delete Data
def delete_underrated_movie(title, year, rating, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.Table('Movies')

    try:
        response = table.delete_item(
            Key={
                'year': year,
                'title': title
            },
            ConditionExpression="info.rating <= :val",
            ExpressionAttributeValues={
                ":val": Decimal(rating)
            }
        )
    except ClientError as e:
        if e.response['Error']['Code'] == "ConditionalCheckFailedException":
            print(e.response['Error']['Message'])
        else:
            raise
    else:
        return response


if __name__ == '__main__':
    print("Attempting a conditional delete...")
    delete_response = delete_underrated_movie("The Big New Movie", 2015, 5.5,dynamodb)
    if delete_response:
        print("Delete movie succeeded:")
        #pprint(delete_response, sort_dicts=False)
        pprint(delete_response)

Attempting a conditional delete...
Delete movie succeeded:
{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '2',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Mon, 03 May 2021 00:33:36 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '2745614147',
                                      'x-amzn-requestid': 'M21EMB5MA41E3L0GKM6Q4032CFVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'M21EMB5MA41E3L0GKM6Q4032CFVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


In [112]:
movie = get_movie("The Big New Movie", 2015,dynamodb)
if movie:
    print("Get movie succeeded:")
    #pprint(movie, sort_dicts=False)
    pprint(movie)

KeyError: 'Item'